#### LangSmith Starter

# LangSmith Starter

LangSmith is an advanced open-source library tailored for AI Engineers, focusing on enabling seamless development and deployment of language-based AI systems. With an emphasis on modularity and interoperability, LangSmith simplifies the integration of natural language processing (NLP) capabilities into AI workflows, offering intuitive tools for fine-tuning, optimization, and interaction. It empowers developers to switch effortlessly between language models, customize pipelines, and deploy scalable, production-ready applications with minimal friction.

Super important note, you need to run LangSmith on more then 0.1.45 and be using the current API keys and not the legacy keys, if you do not do this you will recieve a 401 error message saying the token you are using is invalid.

This below is the enviroment data for enabling tracing, very important that you do not mess around with this as even replacing a singular character can break and then you will not be able to trace anything.

In [40]:
import os
from dotenv import load_dotenv
from getpass import getpass

load_dotenv(dotenv_path=".env")

True

For this tutorial we will be using OpenAI, however this is basically the exact same for llama just with a few changes.

In [41]:
from langchain_ollama.chat_models import ChatOllama

model_name = "qwen:0.5b"

# initialize one LLM with temperature 0.0, this makes the LLM more deterministic
llm = ChatOllama(temperature=0.0, model=model_name)

Langsmith by default will trace all the usual langchain based LLM calls, which in the tutorials case is super useful as we won't have to edit much code to get traces working.

In [42]:
llm.invoke("hello")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'qwen:0.5b', 'created_at': '2025-06-12T09:01:11.711909Z', 'done': True, 'done_reason': 'stop', 'total_duration': 165009000, 'load_duration': 39841100, 'prompt_eval_count': 9, 'prompt_eval_duration': 34181400, 'eval_count': 10, 'eval_duration': 90110100, 'model_name': 'qwen:0.5b'}, id='run--73b4fddb-c1ab-44e5-9791-c6001a99f19e-0', usage_metadata={'input_tokens': 9, 'output_tokens': 10, 'total_tokens': 19})

This is settup for the non-langchain function.

In [43]:
import ollama

client = ollama

Now we want to use a non langchain related function, and these are not automatically traced by langsmith, so instead, we have to add the traceable decorator.

In [44]:
from langsmith import traceable

@traceable
def generate_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat(
        model = model_name,
        messages = complete_messages,
    )

In [45]:
generate_response("How are you today?")

ChatResponse(model='qwen:0.5b', created_at='2025-06-12T09:01:14.2455124Z', done=True, done_reason='stop', total_duration=656486800, load_duration=44848800, prompt_eval_count=22, prompt_eval_duration=61308000, eval_count=56, eval_duration=549048200, message=Message(role='assistant', content="As an AI language model, I don't have emotions or feelings of happiness. However, I am constantly learning and improving my abilities to assist users in various ways. So, while I don't experience happiness, I am always working towards providing the best possible assistance to users.", thinking=None, images=None, tool_calls=None))

We can also add data into the traceable by editing its paramaters, this includes metadata, and what we will change, the function name.

In [46]:
from langsmith import traceable

@traceable(name="Qwen Response")
def generate_second_response(question: str):

    complete_messages = [
    {"role": "system", "content": "You are a happy assistant"},
    {"role": "user", "content": question}
    ]

    return client.chat(
        model = model_name,
        messages = complete_messages,
    )

In [47]:
generate_second_response("How are you today?")

ChatResponse(model='qwen:0.5b', created_at='2025-06-12T09:01:15.6714499Z', done=True, done_reason='stop', total_duration=508944800, load_duration=38948800, prompt_eval_count=22, prompt_eval_duration=22289000, eval_count=40, eval_duration=444729900, message=Message(role='assistant', content="As an AI language model, I don't have feelings or emotions. However, I am always ready and ready to assist you in any way possible. So, how can I help you today?", thinking=None, images=None, tool_calls=None))

Now you can view all of this over at the LangSmith website, under the tracing projects.